# Plot breach data

Using this script you can plot data regarding breaches in a 3Di model. Plots include breach discharge, velocity, width, depth, waterlevel on both sides of the breach.

- Author: Emiel Verstegen, 2019

- email: emiel.verstegen@nelen-schuurmans.nl

In [ ]:
#import libraries, make sure you have them installed on your Python environment
from threedigrid.admin.gridresultadmin import GridH5ResultAdmin
from threedigrid.admin.gridadmin import GridH5Admin
import os
import matplotlib.pyplot as plt
import numpy as np
import math
from pyproj import Proj, transform
from google_drive_downloader import GoogleDriveDownloader as gdd #Needs installing using pip, unlike the others

In [ ]:
#download testdata and unzip in testdata directory (https://drive.google.com/open?id=1xNhMZnWKxFP7on1reXxecq7XdTuJCSa2)


gdd.download_file_from_google_drive(file_id='1xNhMZnWKxFP7on1reXxecq7XdTuJCSa2',
                                    dest_path='./testdata/bergermeer.zip',
                                    unzip=True)

In [ ]:
#Define location of raw results and gridadministration
result_path = 'testdata/'
nc = os.path.join(result_path,'results_3di.nc')
f = os.path.join(result_path,'gridadmin.h5')

#Load files into gr object
ga = GridH5Admin(f)
gr = GridH5ResultAdmin(f,nc)

In [ ]:
#subset only 1D nodes
breaches = gr.breaches
breach_lines = gr.lines.filter(kcu__eq=55)

In [ ]:
breach_idx = breaches.levl[1:]

#Loop over all breaches in the model (in the testdata there is only 1 breach)
for breach_id in breach_idx:
    print("Breach flowline id: {}".format(breach_id))
    breach =  gr.breaches.filter(levl__eq=breach_id)
    flowline = gr.lines.filter(id__eq=breach_id)
    
    #extract the breach depth and width from the results
    depth = breach.timeseries(start_time=1,end_time=gr.nodes.timestamps[-1]).breach_depth
    width = breach.timeseries(start_time=1,end_time=gr.nodes.timestamps[-1]).breach_width
    
    #extract the flowline discharge and velocity from the results
    discharge = flowline.timeseries(start_time=1,end_time=gr.nodes.timestamps[-1]).q
    velocity = flowline.timeseries(start_time=1,end_time=gr.nodes.timestamps[-1]).u1
    
    #select the waterlevel nodes at the beginning and end of the breach-flowline
    start_node_id = flowline.line[0][0]
    end_node_id = flowline.line[1][0]
    start_node = gr.nodes.filter(id__eq=start_node_id)
    end_node = gr.nodes.filter(id__eq=end_node_id)
    
    #extract the waterlevel at both sides of the breach
    wl_startnode = start_node.timeseries(start_time=1,end_time=gr.nodes.timestamps[-1]).s1
    wl_endnode = end_node.timeseries(start_time=1,end_time=gr.nodes.timestamps[-1]).s1
    
    #extract the timestamps of the simulation
    ts = gr.nodes.timestamps[1:]
    
    #Replace -9999 values as NaN
    nodatavalue = -9999
    depth[depth==nodatavalue]=np.nan
    width[width==nodatavalue]=np.nan
    discharge[discharge==nodatavalue]=np.nan
    velocity[velocity==nodatavalue]=np.nan
    wl_startnode[wl_startnode==nodatavalue]=np.nan
    wl_endnode[wl_endnode==nodatavalue]=np.nan

    fig = plt.figure(figsize=(20,10))
    ax1 = plt.subplot2grid((2, 2), (0, 0))
    ax2 = plt.subplot2grid((2, 2), (0, 1))
    ax3 = plt.subplot2grid((2, 2), (1, 0))
    ax4 = plt.subplot2grid((2, 2), (1, 1))

    fig.suptitle('Breach with id {}'
                 .format(breach_id,))
    
    ax1.plot(ts, discharge)
    ax1.set_title("Discharge [m3/s]")
    
    ax2.plot(ts, velocity)
    ax2.set_title("Velocity [m/s]")
    
    ax3.plot(ts,wl_startnode)
    ax3.plot(ts,wl_endnode)
    ax3.set_title("Water levels [m+MSL]")
    
    ax4.plot(ts,width)
    ax4.set_title("Breach width [m]")